In [1]:
import pandas as pd
import numpy as np

import itertools

import sklearn
import scipy

from scipy.spatial.distance import pdist
from sklearn.svm import SVC, SVR
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold

import xgboost as xgb

print(pd.__version__)
#1.3.3
print(np.__version__)
#1.20.3
print(sklearn.__version__)
#0.24.2
print(scipy.__version__)
#1.7.1
print(xgb.__version__)
#1.4.2


1.3.3
1.20.3
0.24.2
1.7.1
1.4.2


In [2]:
tested_odor_desc = pd.read_csv("./compiled_desc_resp/filtered_tested_desc.csv", index_col = 0)
tested_odor_resp = pd.read_csv("./compiled_desc_resp/compiled_odor_sigResp_wide.csv", index_col = 0)

In [3]:
feature_df = pd.DataFrame()

for feature in tested_odor_desc.columns:
    df = tested_odor_desc.loc[:,feature].copy(deep=True)
    unique_value_count = df.nunique()
    zero_count = df[df == 0].shape[0]
    if (zero_count > 0) & (unique_value_count < 52):
        feature_df = pd.concat([feature_df, df], axis = 1)

feature_df_binary = pd.DataFrame()

for feature in feature_df.columns:
    df = feature_df.loc[:,feature].copy(deep=True)
    df[df != 0] = 1
    feature_df_binary = pd.concat([feature_df_binary, df], axis = 1)

In [4]:
feature_df_binary = feature_df_binary.loc[:,feature_df_binary.sum(axis = 0) > 1]
feature_df_binary = feature_df_binary.loc[:,feature_df_binary.sum(axis = 0) < 51]

In [5]:
print(feature_df_binary.shape)
#(52, 763)

(52, 763)


# SVC

In [10]:
#Center-scale function for numpy
def np_normalize(input_vector):
    return((input_vector - input_vector.mean())/((input_vector - input_vector.mean()).std(ddof=1)))

In [6]:
#svc_param_grid = {'C': [0.1, 1, 10, 100, 1000],
#                  'random_state': [42],
#                  'probability': [True],
#                  'kernel': ['linear']}
#
#feature_predictions = pd.DataFrame()
#feat_auc = {}
#
#x = np.array(tested_odor_resp.copy())
#x_scaled = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=x)
#
#counter = 0
#
#for feature in feature_df_binary.columns:
#    #Print counter to keep track of which feature number you are currently on
#    counter += 1
#    print(counter)
#    #Set up whether or not to do CV to define hyper parameters
#    if (feature_df_binary.loc[:,feature].sum() > 2) & (feature_df_binary.loc[:,feature].sum() < 50):
#        cv_var = 2
#    else:
#        cv_var = 1
#    y = feature_df_binary.loc[:,feature]
#    compile_probs = []
#    for i in range(x_scaled.shape[0]):
#        x_test = x_scaled[i].reshape(1,375)
#        x_train = np.delete(arr = x, obj = i, axis = 0)
#        x_train_scaled = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=x_train)
#        y_test = y.iloc[i]
#        y_train = y.drop(index=y.index[i])
#        if cv_var == 2:
#            grid = GridSearchCV(SVC(), svc_param_grid, refit = True, verbose = 0, n_jobs = 20,
#                                cv = StratifiedKFold(n_splits=2, random_state=42, shuffle=True))
#            grid.fit(x_train_scaled, y_train)
#            probas_ = grid.best_estimator_.predict_proba(x_test)
#        else:
#            grid = SVC(random_state=42, probability=True, kernel='linear', C = 1)
#            grid.fit(x_train_scaled, y_train)
#            probas_ = grid.predict_proba(x_test)
#        compile_probs.append(probas_[:,1][0])
#    feat_pred = pd.concat([y.reset_index(drop=True), pd.DataFrame(compile_probs)], axis =1)
#    feat_pred.columns = [y.name+"_true",y.name+"_pred"]
#    #Compile true out & prediction probabilities
#    feature_predictions = pd.concat([feature_predictions, feat_pred], axis = 1)
#    #Compute AUROC
#    fpr, tpr, thresholds = roc_curve(feat_pred.iloc[:,0], feat_pred.iloc[:,1])
#    roc_auc = auc(fpr, tpr)
#    feat_auc[feature] = roc_auc
    
#feature_predictions.to_csv("./descriptor_models/svc_probabilities.csv")
#classifier_auroc = pd.DataFrame.from_dict(feat_auc, orient = 'Index').reset_index(drop=False)
#classifier_auroc.columns = ['feature','auroc']
#classifier_auroc.to_csv("./descriptor_models/svc_auroc.csv")
svc_predictions = pd.read_csv("./descriptor_models/svc_probabilities.csv", index_col = 0)
svc_auroc = pd.read_csv("./descriptor_models/svc_auroc.csv", index_col = 0)

In [7]:
#compile_feat_predictions = pd.DataFrame()
#for i in range(0,svc_predictions.shape[1],2):
#    feat_name = svc_predictions.iloc[:,i].name
#    feat_name = feat_name.removesuffix("_true")
#    true_val = svc_predictions.iloc[:,i].values
#    pred_val = svc_predictions.iloc[:,i+1].values
#    temp_df = pd.DataFrame([true_val, pred_val]).transpose()
#    temp_df['feature'] = feat_name
#    compile_feat_predictions = pd.concat([compile_feat_predictions, temp_df])
#compile_feat_predictions.columns = ['true','prediction','feature']
#compile_feat_predictions = compile_feat_predictions.reset_index(drop=True)
#compile_feat_predictions.to_csv("./descriptor_models/svc_probabilities_long.csv")

In [8]:
svc_pass = svc_auroc[svc_auroc['auroc'] > 0.75]
tested_odor_desc_svc_pass = tested_odor_desc.loc[:,tested_odor_desc.columns.isin(svc_pass['feature'])]
print(tested_odor_desc_svc_pass.shape)
#(52, 147)

(52, 147)


# SVR

In [ ]:
#svr_param_grid = {'C': [0.1, 1, 10, 100, 1000],
#                  'kernel': ['linear']}
#
#linear_feature_regression = pd.DataFrame()
#
#x = np.array(tested_odor_resp.copy())
#x_scaled = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=x)
#
#for feature in tested_odor_desc_svc_pass.columns:  
#    if (tested_odor_desc_svc_pass[tested_odor_desc_svc_pass.loc[:,feature] != 0].shape[0] > 2) & \
#    (tested_odor_desc_svc_pass[tested_odor_desc_svc_pass.loc[:,feature] != 0].shape[0] < 50):
#        cv_var = 2
#    else:
#        cv_var = 1
#    y = tested_odor_desc_svc_pass.loc[:,feature]
#    compile_val = []
#    for i in range(x_scaled.shape[0]):
#        x_test = x_scaled[i].reshape(1,375)
#        x_train = np.delete(arr = x, obj = i, axis = 0)
#        x_train_scaled = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=x_train)
#        y_test = y.iloc[i]
#        y_train = y.drop(index=y.index[i])
#        if cv_var == 2:
#            grid = GridSearchCV(SVR(), svr_param_grid, refit = True, verbose = 0, n_jobs = 20,
#                                cv = KFold(n_splits=2, random_state=42, shuffle=True))
#            grid.fit(x_train_scaled, y_train)
#            y_pred = grid.best_estimator_.predict(x_test)
#        else:
#            grid = SVR(kernel='linear', C = 1)
#            grid.fit(x_train_scaled, y_train)
#            y_pred = grid.predict(x_test)
#        compile_val.append(y_pred)
#    feat_reg = pd.concat([y.reset_index(drop=True), pd.DataFrame(compile_val)], axis =1)
#    feat_reg.columns = [y.name+"_true",y.name+"_pred"]
#    #Compile true out & predictions
#    linear_feature_regression = pd.concat([linear_feature_regression, feat_reg], axis = 1)
#
#s = '_'
#linear_regression_dict = {}
#for i in range(0, linear_feature_regression.shape[1], 2):
#    name = linear_feature_regression.columns[i]
#    name = name.split('_')
#    name = name[:len(name)-1]
#    name = s.join(name)
#    pearson_r = np.corrcoef(linear_feature_regression.iloc[:,i+0],linear_feature_regression.iloc[:,i+1])[0,1]
#    linear_regression_dict[name] = np.corrcoef(linear_feature_regression.iloc[:,i+0],linear_feature_regression.iloc[:,i+1])[0,1]
#
#linear_regression_df = pd.DataFrame.from_dict(linear_regression_dict, orient='Index')
#linear_regression_df = linear_regression_df.reset_index(drop=False)
#linear_regression_df.columns = ['feature','pearson_r']
#well_predicted_feats = linear_regression_df[linear_regression_df['pearson_r'] > 0.5]
#well_predicted_desc = tested_odor_desc.loc[:,tested_odor_desc.columns.isin(well_predicted_feats['feature'])]
#linear_feature_regression.to_csv("./descriptor_models/svr_values.csv")
#well_predicted_desc.to_csv("./compiled_desc_resp/well_predicted_desc.csv")
#well_predicted_feats.to_csv("./descriptor_models/svr_r.csv")


In [3]:
svr_predictions = pd.read_csv("./descriptor_models/svr_values.csv", index_col = 0)
svr_r = pd.read_csv("./descriptor_models/svr_r.csv", index_col = 0)
well_predicted_desc = pd.read_csv("./compiled_desc_resp/well_predicted_desc.csv", index_col = 0)

In [39]:
#compile_feat_predictions = pd.DataFrame()
#for i in range(0,svr_predictions.shape[1],2):
#    feat_name = svr_predictions.iloc[:,i].name
#    feat_name = feat_name.removesuffix("_true")
#    true_val = svr_predictions.iloc[:,i].values
#    pred_val = svr_predictions.iloc[:,i+1].values
#    pearson_r = np.corrcoef(true_val, pred_val)[0,1]
#    temp_df = pd.DataFrame([true_val, pred_val]).transpose()
#    temp_df['pearson_r'] = pearson_r
#    temp_df['feature'] = feat_name
#    compile_feat_predictions = pd.concat([compile_feat_predictions, temp_df])
#compile_feat_predictions.columns = ['true','prediction','pearson_r','feature']
#compile_feat_predictions = compile_feat_predictions.reset_index(drop=True)
#compile_feat_predictions.to_csv("./descriptor_models/svr_probabilities_long.csv")

# Calculate distances of optimized descriptors

In [17]:
#Center-scale function for pandas
def normalize(x):
    return (x - x.mean())/(x - x.mean()).std()

In [19]:
##Load data
#tested_odor_desc = pd.read_csv("./compiled_desc_resp/well_predicted_desc.csv", index_col = 0)
#tested_odor_resp = pd.read_csv("./compiled_desc_resp/compiled_odor_sigResp_wide.csv", index_col = 0)
##Normalize data
#norm_tested_odor_desc = tested_odor_desc.transform(normalize)
#norm_tested_odor_resp = tested_odor_resp.transform(normalize)
##Calculate distances
#odor_combns2 = pd.DataFrame(itertools.combinations(list(tested_odor_resp.index), 2))
#norm_tested_resp_dist_euc = pd.DataFrame(pdist(norm_tested_odor_resp, 'euclidean'))
#norm_tested_desc_dist_euc = pd.DataFrame(pdist(norm_tested_odor_desc, 'euclidean'))
#norm_tested_resp_dist_corr = pd.DataFrame(pdist(norm_tested_odor_resp, 'correlation'))
#norm_tested_desc_dist_corr = pd.DataFrame(pdist(norm_tested_odor_desc, 'correlation'))
#norm_tested_resp_dist_cos = pd.DataFrame(pdist(norm_tested_odor_resp, 'cosine'))
#norm_tested_desc_dist_cos = pd.DataFrame(pdist(norm_tested_odor_desc, 'cosine'))
##Combine distances
#compiled_distances = pd.concat([odor_combns2, norm_tested_resp_dist_euc,
#                                norm_tested_desc_dist_euc,norm_tested_resp_dist_corr,
#                                norm_tested_desc_dist_corr,norm_tested_resp_dist_cos,
#                                norm_tested_desc_dist_cos], axis = 1)
#compiled_distances.columns = ['odor1','odor2',
#                              'response_euc_distance','feature_euc_distance',
#                              'response_corr_distance','feature_corr_distance',
#                              'response_cos_distance','feature_cos_distance']
#compiled_distances.to_csv("./compiled_desc_resp/optimized_descriptors_distances.csv")
opt_compiled_distances = pd.read_csv("./compiled_desc_resp/optimized_descriptors_distances.csv", index_col = 0)


# XGBoost - all descs

In [542]:
#Center-scale function
def normalize(x):
    return (x - x.mean())/(x - x.mean()).std()

#Center-scale function for numpy
def np_normalize(input_vector):
    return((input_vector - input_vector.mean())/((input_vector - input_vector.mean()).std(ddof=1)))

#Calculate distances between features
def feature_distances(input_vector):
    modified_vector = input_vector.reshape(-1,1)
    vector_distances = pdist(modified_vector, 'euclidean')
    return vector_distances

#Center-scale function for shuffled data
def scale_shuffled(input_vector):
    if input_vector.var() == 0:
        input_vector.values[:] = 0
    else:
        input_vector = input_vector.transform(normalize)
    return input_vector


In [543]:
#Load data
tested_odor_desc = pd.read_csv("./compiled_desc_resp/filtered_tested_desc.csv", index_col = 0)
tested_odor_resp = pd.read_csv("./compiled_desc_resp/compiled_odor_sigResp_wide.csv", index_col = 0)
#Normalize data
norm_tested_odor_desc = tested_odor_desc.transform(normalize)
norm_tested_odor_resp = tested_odor_resp.transform(normalize)
#Load shuffled data
shuffled_odor_desc = pd.read_csv("./compiled_desc_resp/shuffled_filtered_tested_desc.csv", index_col = 0)
shuffled_odor_desc.index = tested_odor_desc.index
shuffled_odor_desc.columns = tested_odor_desc.columns
#Normalize shuffled data
norm_shuffled_odor_desc = shuffled_odor_desc.transform(scale_shuffled)
norm_shuffled_odor_desc.columns = norm_tested_odor_desc.columns

In [544]:
#Convert base data to numpy array for faster processing
tested_odor_desc = np.array(tested_odor_desc)
tested_odor_resp = np.array(tested_odor_resp)
shuffled_odor_desc = np.array(shuffled_odor_desc)
#Convert normalized data to numpy array for faster processing
norm_tested_odor_desc = np.array(norm_tested_odor_desc)
norm_tested_odor_resp = np.array(norm_tested_odor_resp)
norm_shuffled_odor_desc = np.array(norm_shuffled_odor_desc)

In [ ]:
#xgb_predictions = pd.DataFrame()
#
#for i, (o1, o2) in enumerate(itertools.combinations(range(tested_odor_desc.shape[0]), 2)):
#    print(i)
#    temp_predictions = pd.DataFrame()
#    #Setup/reset models
#    Euc_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#    cor_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#    cos_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#    #Setup testing data
#    norm_testing_desc = norm_tested_odor_desc[[o1,o2]]
#    norm_testing_resp = norm_tested_odor_resp[[o1, o2]]
#    shuffled_testing_desc = norm_shuffled_odor_desc[[o1, o2]]
#    #Setup training data
#    training_desc = np.delete(tested_odor_desc, [o1,o2], axis = 0)
#    training_resp = np.delete(tested_odor_resp, [o1,o2], axis = 0)
#    #Correct numerical instabilities
#    if (o1 == 19) & (o2 == 48):
#        training_desc[:,1658] = 0
#    #Remove features with zero variance in training data
#    if (training_desc.var(ddof = 1, axis = 0) == 0).any():
#        #Identify zero variance column numbers
#        zero_var_cols = np.where((training_desc.var(ddof = 1,axis = 0) == 0))
#        #Drop zero variance features from training/testing/shuffled datasets
#        training_desc = np.delete(training_desc, zero_var_cols, axis = 1)
#        norm_testing_desc = np.delete(norm_testing_desc, zero_var_cols, axis = 1)
#        shuffled_testing_desc = np.delete(shuffled_testing_desc, zero_var_cols, axis = 1)
#    #Normalize training data in absence of test set
#    norm_training_desc = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=training_desc)
#    norm_training_resp = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=training_resp)
#    #Calculate descriptor distances in train and test set
#    norm_training_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=norm_training_desc)
#    norm_testing_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=norm_testing_desc)
#    shuffled_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=shuffled_testing_desc)
#    #Calculate response distances
#    norm_training_resp_dist_Euc = pdist(norm_training_resp, 'euclidean')
#    norm_testing_resp_dist_Euc = pdist(norm_testing_resp, 'euclidean')
#    #Calculate correlation response distances
#    norm_training_resp_dist_cor = pdist(norm_training_resp, 'correlation')
#    norm_testing_resp_dist_cor = pdist(norm_testing_resp, 'correlation')
#    #Calculate cosine response distances
#    norm_training_resp_dist_cos = pdist(norm_training_resp, 'cosine')
#    norm_testing_resp_dist_cos = pdist(norm_testing_resp, 'cosine')
#    #Fit models
#    Euc_model.fit(norm_training_desc_dist, norm_training_resp_dist_Euc)
#    cor_model.fit(norm_training_desc_dist, norm_training_resp_dist_cor)
#    cos_model.fit(norm_training_desc_dist, norm_training_resp_dist_cos)
#    #Use models to predict intact distances
#    ypred_Euc = Euc_model.predict(norm_testing_desc_dist)
#    ypred_cor = cor_model.predict(norm_testing_desc_dist)
#    ypred_cos = cos_model.predict(norm_testing_desc_dist)
#    #Use models to predict shuffled distances
#    ypred_Euc_shuffled = Euc_model.predict(shuffled_desc_dist)
#    ypred_cor_shuffled = cor_model.predict(shuffled_desc_dist)
#    ypred_cos_shuffled = cos_model.predict(shuffled_desc_dist)
#    #Compile predictions
#    temp_predictions = pd.concat([pd.DataFrame(norm_testing_resp_dist_Euc),
#                                  pd.DataFrame(norm_testing_resp_dist_cor),
#                                  pd.DataFrame(norm_testing_resp_dist_cos),
#                                  pd.DataFrame(ypred_Euc),
#                                  pd.DataFrame(ypred_cor),
#                                  pd.DataFrame(ypred_cos),
#                                  pd.DataFrame(ypred_Euc_shuffled),
#                                  pd.DataFrame(ypred_cor_shuffled),
#                                  pd.DataFrame(ypred_cos_shuffled)],
#                                  axis = 1)
#    xgb_predictions = pd.concat([xgb_predictions, temp_predictions], axis = 0)
#    
#xgb_predictions.columns = ['ytrue_Euc','ytrue_cor','ytrue_cos',
#                           'ypred_Euc','ypred_cor','ypred_cos',
#                           'ypred_Euc_shuffled','ypred_cor_shuffled','ypred_cos_shuffled'
#                           ]
#
##Create new models for full fitting evaluation
#Euc_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#cor_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#cos_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
##Calculate full fit response distances
#norm_tested_odor_resp_dist_Euc = pd.DataFrame(pdist(norm_tested_odor_resp, 'euclidean'))
#norm_tested_odor_resp_dist_cor = pd.DataFrame(pdist(norm_tested_odor_resp, 'correlation'))
#norm_tested_odor_resp_dist_cos = pd.DataFrame(pdist(norm_tested_odor_resp, 'cosine'))
##Calculate full fit feature Euclidean distances
#norm_tested_odor_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=norm_tested_odor_desc)
##Fit models
#Euc_model.fit(norm_tested_odor_desc_dist, norm_tested_odor_resp_dist_Euc)
#cor_model.fit(norm_tested_odor_desc_dist, norm_tested_odor_resp_dist_cor)
#cos_model.fit(norm_tested_odor_desc_dist, norm_tested_odor_resp_dist_cos)      
##Use full fit models to make predictions
#full_fit_pred_Euc = pd.DataFrame(Euc_model.predict(norm_tested_odor_desc_dist))
#full_fit_pred_cor = pd.DataFrame(cor_model.predict(norm_tested_odor_desc_dist))
#full_fit_pred_cos = pd.DataFrame(cos_model.predict(norm_tested_odor_desc_dist))
##Compile full fit
#full_fit_out = pd.concat([norm_tested_odor_resp_dist_Euc,
#                          norm_tested_odor_resp_dist_cor,
#                          norm_tested_odor_resp_dist_cos,
#                          full_fit_pred_Euc,
#                          full_fit_pred_cor,
#                          full_fit_pred_cos], axis = 1)
#full_fit_out.columns = ['fullFit_ytrue_Euc','fullFit_ytrue_cor','fullFit_ytrue_cos',
#                        'fullFit_ypred_Euc','fullFit_ypred_cor','fullFit_ypred_cos']
##Save files
#xgb_predictions.to_csv("./descriptor_models/xgb_LOOCV_allDesc_variousMetrics.csv")
#full_fit_out.to_csv("./descriptor_models/xgb_fullFit_allDesc_variousMetrics.csv")

# XGBoost - optimized descs

In [2]:
#Center-scale function
def normalize(x):
    return (x - x.mean())/(x - x.mean()).std()

#Center-scale function for numpy
def np_normalize(input_vector):
    return((input_vector - input_vector.mean())/((input_vector - input_vector.mean()).std(ddof=1)))

#Calculate distances between features
def feature_distances(input_vector):
    modified_vector = input_vector.reshape(-1,1)
    vector_distances = pdist(modified_vector, 'euclidean')
    return vector_distances

#Center-scale function for shuffled data
def scale_shuffled(input_vector):
    if input_vector.var() == 0:
        input_vector.values[:] = 0
    else:
        input_vector = input_vector.transform(normalize)
    return input_vector

In [3]:
#Load data
well_predicted_desc = pd.read_csv("./compiled_desc_resp/well_predicted_desc.csv", index_col = 0)
tested_odor_resp = pd.read_csv("./compiled_desc_resp/compiled_odor_sigResp_wide.csv", index_col = 0)
tested_odor_desc = well_predicted_desc.copy(deep=True)
#Normalize data
norm_tested_odor_desc = tested_odor_desc.transform(normalize)
norm_tested_odor_resp = tested_odor_resp.transform(normalize)
#Load shuffled data
shuffled_odor_desc = pd.read_csv("./compiled_desc_resp/shuffled_filtered_tested_optDesc.csv", index_col = 0)
shuffled_odor_desc.index = tested_odor_desc.index
#Normalize shuffled data, no columns display numerical instabilities
norm_shuffled_odor_desc = shuffled_odor_desc.transform(scale_shuffled)
norm_shuffled_odor_desc.columns = norm_tested_odor_desc.columns

In [580]:
#Convert base data to numpy array for faster processing
tested_odor_desc = np.array(tested_odor_desc)
tested_odor_resp = np.array(tested_odor_resp)
shuffled_odor_desc = np.array(shuffled_odor_desc)
#Convert normalized data to numpy array for faster processing
norm_tested_odor_desc = np.array(norm_tested_odor_desc)
norm_tested_odor_resp = np.array(norm_tested_odor_resp)
norm_shuffled_odor_desc = np.array(norm_shuffled_odor_desc)

In [ ]:
#xgb_predictions = pd.DataFrame()
#
#for i, (o1, o2) in enumerate(itertools.combinations(range(tested_odor_desc.shape[0]), 2)):
#    print(i)
#    temp_predictions = pd.DataFrame()
#    #Setup/reset models
#    Euc_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#    cor_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#    cos_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#    #Setup testing data
#    norm_testing_desc = norm_tested_odor_desc[[o1,o2]]
#    norm_testing_resp = norm_tested_odor_resp[[o1, o2]]
#    shuffled_testing_desc = norm_shuffled_odor_desc[[o1, o2]]
#    #Setup training data
#    training_desc = np.delete(tested_odor_desc, [o1,o2], axis = 0)
#    training_resp = np.delete(tested_odor_resp, [o1,o2], axis = 0)
#    #Remove features with zero variance in training data
#    if (training_desc.var(ddof = 1, axis = 0) == 0).any():
#        #Identify zero variance column numbers
#        zero_var_cols = np.where((training_desc.var(ddof = 1,axis = 0) == 0))
#        #Drop zero variance features from training/testing/shuffled datasets
#        training_desc = np.delete(training_desc, zero_var_cols, axis = 1)
#        norm_testing_desc = np.delete(norm_testing_desc, zero_var_cols, axis = 1)
#        shuffled_testing_desc = np.delete(shuffled_testing_desc, zero_var_cols, axis = 1)
#    #Normalize training data in absence of test set
#    norm_training_desc = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=training_desc)
#    norm_training_resp = np.apply_along_axis(func1d = np_normalize, axis = 0, arr=training_resp)
#    #Calculate descriptor distances in train and test set
#    norm_training_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=norm_training_desc)
#    norm_testing_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=norm_testing_desc)
#    shuffled_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=shuffled_testing_desc)
#    #Calculate response distances
#    norm_training_resp_dist_Euc = pdist(norm_training_resp, 'euclidean')
#    norm_testing_resp_dist_Euc = pdist(norm_testing_resp, 'euclidean')
#    #Calculate correlation response distances
#    norm_training_resp_dist_cor = pdist(norm_training_resp, 'correlation')
#    norm_testing_resp_dist_cor = pdist(norm_testing_resp, 'correlation')
#    #Calculate cosine response distances
#    norm_training_resp_dist_cos = pdist(norm_training_resp, 'cosine')
#    norm_testing_resp_dist_cos = pdist(norm_testing_resp, 'cosine')
#    #Fit models
#    Euc_model.fit(norm_training_desc_dist, norm_training_resp_dist_Euc)
#    cor_model.fit(norm_training_desc_dist, norm_training_resp_dist_cor)
#    cos_model.fit(norm_training_desc_dist, norm_training_resp_dist_cos)
#    #Use models to predict intact distances
#    ypred_Euc = Euc_model.predict(norm_testing_desc_dist)
#    ypred_cor = cor_model.predict(norm_testing_desc_dist)
#    ypred_cos = cos_model.predict(norm_testing_desc_dist)
#    #Use models to predict shuffled distances
#    ypred_Euc_shuffled = Euc_model.predict(shuffled_desc_dist)
#    ypred_cor_shuffled = cor_model.predict(shuffled_desc_dist)
#    ypred_cos_shuffled = cos_model.predict(shuffled_desc_dist)
#    #Compile predictions
#    temp_predictions = pd.concat([pd.DataFrame(norm_testing_resp_dist_Euc),
#                                  pd.DataFrame(norm_testing_resp_dist_cor),
#                                  pd.DataFrame(norm_testing_resp_dist_cos),
#                                  pd.DataFrame(ypred_Euc),
#                                  pd.DataFrame(ypred_cor),
#                                  pd.DataFrame(ypred_cos),
#                                  pd.DataFrame(ypred_Euc_shuffled),
#                                  pd.DataFrame(ypred_cor_shuffled),
#                                  pd.DataFrame(ypred_cos_shuffled)],
#                                  axis = 1)
#    xgb_predictions = pd.concat([xgb_predictions, temp_predictions], axis = 0)
#    
#xgb_predictions.columns = ['ytrue_Euc','ytrue_cor','ytrue_cos',
#                           'ypred_Euc','ypred_cor','ypred_cos',
#                           'ypred_Euc_shuffled','ypred_cor_shuffled','ypred_cos_shuffled'
#                           ]
#
##Create new models for full fitting evaluation
#Euc_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#cor_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
#cos_model = xgb.XGBRegressor(random_state = 42, n_jobs = -1)
##Calculate full fit response distances
#norm_tested_odor_resp_dist_Euc = pd.DataFrame(pdist(norm_tested_odor_resp, 'euclidean'))
#norm_tested_odor_resp_dist_cor = pd.DataFrame(pdist(norm_tested_odor_resp, 'correlation'))
#norm_tested_odor_resp_dist_cos = pd.DataFrame(pdist(norm_tested_odor_resp, 'cosine'))
##Calculate full fit feature Euclidean distances
#norm_tested_odor_desc_dist = np.apply_along_axis(func1d = feature_distances, axis = 0, arr=norm_tested_odor_desc)
##Fit models
#Euc_model.fit(norm_tested_odor_desc_dist, norm_tested_odor_resp_dist_Euc)
#cor_model.fit(norm_tested_odor_desc_dist, norm_tested_odor_resp_dist_cor)
#cos_model.fit(norm_tested_odor_desc_dist, norm_tested_odor_resp_dist_cos)      
##Use full fit models to make predictions
#full_fit_pred_Euc = pd.DataFrame(Euc_model.predict(norm_tested_odor_desc_dist))
#full_fit_pred_cor = pd.DataFrame(cor_model.predict(norm_tested_odor_desc_dist))
#full_fit_pred_cos = pd.DataFrame(cos_model.predict(norm_tested_odor_desc_dist))
##Compile full fit
#full_fit_out = pd.concat([norm_tested_odor_resp_dist_Euc,
#                          norm_tested_odor_resp_dist_cor,
#                          norm_tested_odor_resp_dist_cos,
#                          full_fit_pred_Euc,
#                          full_fit_pred_cor,
#                          full_fit_pred_cos], axis = 1)
#full_fit_out.columns = ['fullFit_ytrue_Euc','fullFit_ytrue_cor','fullFit_ytrue_cos',
#                        'fullFit_ypred_Euc','fullFit_ypred_cor','fullFit_ypred_cos']
##Save files
#xgb_predictions.to_csv("./descriptor_models/xgb_LOOCV_optDesc_variousMetrics.csv")
#full_fit_out.to_csv("./descriptor_models/xgb_fullFit_optDesc_variousMetrics.csv")